In [7]:
import pandas as pd
import numpy as np
import warnings

# Framework d'Analyse de Portefeuille

## Import & Traitement des Données

### Fonds & Valeurs Liquidatives

In [8]:
class FundData:
    def __init__(self, isin: str, filename: str, file_format: str, date_idx : int, value_idx : int, first_date : str):
        self.isin = isin
        self.filename = filename
        self.file_format = file_format
        self.date_idx = date_idx
        self.value_idx = value_idx
        self.first_date = first_date
        self.df = self.load_data()
    
    def load_data(self):
        data = self.import_data()
        data_filtered = self.filter_columns(data)
        final_data = self.clean_data(data_filtered)
        return final_data

    def import_data(self):
        dict_format = {"xlsx":"excel","csv":"csv"}
        if self.file_format in ["xlsx","csv"]:
            data = getattr(pd,f"read_{dict_format[self.file_format]}")(f"funds/{self.filename}.{self.file_format}")
            return data
        else:
            raise ValueError("Unsupported file type. Use 'xlsx' or 'csv'.")
    
    def filter_columns(self, data):
        data = data[[data.columns[self.date_idx], data.columns[self.value_idx]]] # on garde les colonnes qui nous interessent, ici ce sera les dates et VL.
        data.columns = ["Date", self.isin] # on renomme les colonnes (ISIN du fonds en haut de la colonne des VL)
        return data
    
    def clean_data(self, data_filtered):
        while data_filtered.iloc[0,0] != self.first_date: # tant que la première cellule du dataframe ne correspond pas à la première date de la série
            data_filtered = data_filtered.iloc[1:] # on supprime la première ligne
        data_filtered = data_filtered.dropna(axis="index") # supprime les lignes vides
        data_filtered.Date = pd.to_datetime(data_filtered.Date, infer_datetime_format=True) # format datetime pour les dates
        data_filtered[self.isin] = data_filtered[self.isin].replace(",",".",regex=True).astype(float) # format float pour les VL
        return data_filtered

#### Fonds 1 : JPM America Equity C (acc)
- ISIN : LU0129459060
- Société de gestion : JPM AM
- Actifs : Actions
- Market Cap : Large cap
- Géographie : US
- Devise : US$ 
- Données : 30/08/2019 au 30/08/2024 (daily)
- Format : excel

In [9]:
jpm_vl = FundData(
    isin="LU0129459060",
    filename= "JPMorgan Funds - America Equity Fund",
    file_format= "xlsx",
    date_idx= 0,
    value_idx=1,
    first_date= "30.08.2019",
            )

#### Fonds 2 : Schroder International Selection Fund Global Sustainable Growth C Accumulation USD
- ISIN : LU0557290854
- Société de gestion : Schroder
- Actifs : Actions
- Market Cap : Large & Mid cap
- Géographie : Monde
- Devise : US$ 
- Données : 23/11/2010 au 09/10/2024 (daily)
- Format : csv

In [15]:
schroder_vl = FundData(
    isin="LU0557290854",
    filename= "Schroder International Selection Fund Global Sustainable Growth C Accumulation USD",
    file_format= "csv",
    date_idx= 4,
    value_idx= 5,
    first_date= "09.10.2024",
            )

warnings.filterwarnings('ignore')

#### Fonds 3 : AQR Large Cap Multi-Style Fund
- ISIN : US00203H4956
- Société de gestion : AQR Funds
- Actif : Actions
- Market Cap : Large cap
- Géographie : US
- Devise : US$ 
- Données : 26/03/2013 au 30/09/2024 (daily)
- Format : csv

Remarque : 
Pour ce fonds, différentes parts sont données par le prof, chaque part présente des caractéristiques différentes: 
- La part I : pour les gros investisseurs institutionnels. Minimum d'investissement élevé. Pour ce fonds et cette part, le minimum d'investissement est à 5 millions $.
- La part N : investisseurs institutionnels et privés. Pour ce fonds et cette part, le minimum d'investissement est à 2500$.
- La part R6 : pour les plans de retraites. Pour ce fonds et cette part, le minimum d'investissement est à 50 millions $.
- Les fonds du dessus porte la part C (lower investment minimums and carry a level-load structure) qui se rapproche de la classe N. Pour que nos comparaison soient pertinentes, nous partirons donc sur l'analyse de la classe N du fonds.

In [11]:
aqr_vl = FundData(
    isin="US00203H4956",
    filename= "AQR Large Cap Multi-Style Fund Daily Price History",
    file_format= "csv",
    date_idx= 2,
    value_idx= 6,
    first_date= "26/03/2013 00:00",
            )

In [12]:
funds_vl = pd.merge(pd.merge(jpm_vl.df, schroder_vl.df, on="Date", how="inner"), aqr_vl.df, on="Date", how="inner")
funds_vl

,Date,LU0129459060,LU0557290854,US00203H4956
0,2019-08-30,35.05,218.0506,16.42
1,2019-09-04,34.97,214.5538,16.46
2,2019-09-05,35.47,212.1062,16.72
3,2019-09-09,35.41,222.6949,16.75
4,2019-09-10,35.16,218.6702,16.77
...,...,...,...,...
1084,2024-08-26,77.20,435.9759,20.39
1085,2024-08-27,76.69,434.1793,20.40
1086,2024-08-28,76.83,436.5764,20.33
1087,2024-08-29,76.90,437.2779,20.33
